# **HuggingFace**

기계학습 및 인공지능을 위한 플랫폼과 커뮤니티를 제공하는 프랑스계 미국 회사

자연어처리에 쓰이는 주요 라이브러리
- **transformers**: 트랜스포머 모델의 훈련 및 공유와 관련된 주요 기능 제공
- **datasets**: 데이터셋 처리를 위한 각종 기능 제공
- **evaluate**: 다양한 평가 지표를 손쉽게 계산하는 모듈 제공  
...

# **week9: HuggingFace tutorial**

※ HuggingFace NLP Course의 2과 내용을 재구성  
원본: [Huggingface - Learn - NLP Course - Chapter 2](https://huggingface.co/learn/nlp-course/chapter2/1?fw=pt)

In [ ]:
# !pip install torch transformers -U # 환경에 따라 설치 필요

In [2]:
# import warnings
# from transformers import logging

# 경고 메시지를 무시하고 싶다면 실행
# warnings.filterwarnings(action='ignore')
# logging.set_verbosity_error()

## pipeline

task 수행 과정을 손쉽게 구현해주는 도구  
참고: [HuggingFace Documentations - Transformers - API - Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)  

#### pipeline의 내부 구조

en_chapter2_full_nlp_pipeline.svg

In [3]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis") # = "text-classification"
# default model: "distilbert-base-uncased-finetuned-sst-2-english"
# model 인자로 다른 모델의 경로를 넣으면 해당 모델을 사용할 수 있다.
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

##### **Tokenizer**

In [4]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, # 토큰화할 텍스트
                   padding=True, # 짧은 시퀀스는 pad
                   truncation=True, # 너무 긴 시퀀스는 truncate
                   return_tensors="pt") # PyTorch 텐서 반환
print(inputs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


##### **Model**



en_chapter2_transformer_and_head.svg

In [5]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
# Head가 없는 모델

outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

torch.Size([2, 16, 768])


**Q**. \[ 2, 16, 768 \]은 무엇을 의미하는가?   
**A**. \[ batch_size, seq_len,
 hidden_dim \]을 의미한다.

In [6]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# SequenceClassification Head가 추가된 모델

outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 2])


**Q**. \[ 2, 2 \]는 무엇을 의미하는가?  
**A**. \[ batch_size, num_labels \]를 의미한다.

자연어처리의 다른 task를 처리하는 모델
- ~ForCausalLM
- ~ForMaskedLM
- ~ForSeq2SeqLM
- ~ForTokenClassification
- ~ForNextSentencePrediction

##### **Post Processing**

In [7]:
print(outputs.logits) # logit은 직관적으로 알아보기 어렵다.

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [8]:
import torch

predictions = torch.softmax(outputs.logits, dim=-1) # 확률 분포로 변환
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [9]:
model.config.id2label # 두 확률 중 더 큰 쪽 → 모델의 예측

{0: 'NEGATIVE', 1: 'POSITIVE'}

## Model

※ 모든 사람이 모델을 각자 훈련해서 쓰기에는, 데이터/장비/시간이 부족하다.  
∴ 사전 훈련된 모델을 저장하거나 불러옴으로써, 공유할 수 있어야 한다.

#### 모델 생성/불러오기

In [10]:
# 1. Config로부터 BERT 모델 생성하기
# 랜덤 초기화된 모델 생성
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

In [11]:
# Config의 일부 확인
print("hidden size:", config.hidden_size)
print("intermediate_size:", config.intermediate_size)
print("max_position_embeddings:", config.max_position_embeddings)
print("num_attention_heads:", config.num_attention_heads)
print("num_hidden_layers:", config.num_hidden_layers)

hidden size: 768
intermediate_size: 3072
max_position_embeddings: 512
num_attention_heads: 12
num_hidden_layers: 12


In [12]:
# 2. 사전 훈련된 BERT 모델 불러오기
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [13]:
# 3. 사전 훈련된 모델 자동으로 불러오기
# Config에서 모델이 속한 클래스를 찾아 자동 연결
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")

#### 모델 저장

In [14]:
model.save_pretrained("directory_on_my_computer")

Colab의 경우, 왼쪽의 파일 탭을 눌러 저장된 체크포인트를 확인할 수 있다.

#### 모델 inference

In [15]:
sequences = ["Hello!", "Cool.", "Nice!"]

encoded_sequences = [ # 토큰화 결과
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [16]:
import torch

model_inputs = torch.tensor(encoded_sequences) # PyTorch 텐서로 변환
output = model(model_inputs) # 모델을 텐서를 넣어 call

## Tokenizer

  1. 텍스트를 토큰으로 쪼갠다.
  2. 각 토큰을 맵핑된 정수로 바꾼다.
  3. 모델에게 유용한 다른 input들을 추가한다.
  

#### 다양한 토큰화 방식

1. **Word-based**  
en_chapter2_word_based_tokenization.svg  

단어, 문장부호 단위로 토큰을 분절한다.  
Python의 split() 메소드 등을 통해 간단히 구현 가능하다.
  
단점
- 형태만 다르고 의미가 같은 단어들(e.g. dog/dogs)이 다르게 인코딩되며, 모델은 이러한 단어들의 관계를 모른다.
- 위와 같은 이유로, 단어들을 포착하기 위해 필요한 vocab 수가 너무 많아진다.
- 토크나이저의 유연성이 낮아져, unknown 토큰의 발생 빈도가 높아진다. (= 원본 텍스트의 정보가 소실된다.)

2. **Character-based**  
en_chapter2_character_based_tokenization.svg  

문자 단위로 토큰을 분절한다.  
  
단점
- 텍스트를 처리하는 데 너무 많은 토큰이 필요해진다.
- 많은 언어에서 문자 하나는 의미를 전혀 담지 못한다.

3. **Subword-based**  
en_chapter2_bpe_subword.svg  

단어보다 조금 잘은 '서브워드(subword)' 단위로 토큰을 분절한다.

※ 최신 모델들은 모두 서브워드 토크나이저를 사용한다.
- WordPiece (e.g. BERT)
- Byte-level BPE (e.g. GPT-2)
- SentencePiece BPE (e.g. Llama)
- SentencePiece Unigram (e.g. FLAN-T5)

#### 토크나이저 불러오기/저장

사전훈련된 모델은 훈련 당시의 토크나이저와만 호환되므로, 모델 공유는 토크나이저를 포함해야 한다.

In [17]:
# 1. 모델별 클래스로부터 토크나이저 불러오기
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [18]:
# 2. 토크나이저 자동 불러오기
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [19]:
# 3. 토크나이저 저장하기
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

#### 토큰화 및 인코딩

토큰화(Tokenization)  
sentence → tokens

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


인코딩(Encoding)  
tokens → ids

In [21]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


디코딩(Decoding)  
ids → sentence

In [22]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


#### 특수 토큰(Special Tokens)

In [23]:
sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence) # 토크나이저의 tokenize 메소드 사용
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

model_inputs = tokenizer(sequence) # 토크나이저를 call
print(model_inputs["input_ids"]) # 맨 앞뒤에 특수 토큰의 id 추가

[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119]
[101, 146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119, 102]


In [24]:
print(tokenizer.decode(ids))
print(tokenizer.decode(model_inputs["input_ids"]))

I've been waiting for a HuggingFace course my whole life.
[CLS] I've been waiting for a HuggingFace course my whole life. [SEP]


※ \[CLS\], \[SEP\]: BERT가 사전훈련될 때 입력 문장들에 붙였던 특수 토큰

## 여러 문장 처리하기

#### Batch Encoding

※ 모델은 사실 기본적으로 input_ids의 batch를 입력받는다.


In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor(ids)
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])


IndexError: too many indices for tensor of dimension 1

In [26]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


\[101, 1045, 1005, ... , 102\] - 인코딩  
\[\[101, 1045, 1005, ... , 102\]\] - 인코딩 1개가 들어있는 batch

토크나이저에 문장을 넣으면, 자동으로 batch 단위로 인코딩해준다.

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


#### Padding

토큰화된 문장들을 batch로 묶어 텐서로 변환하려면, 모두 길이가 같아야 한다.

In [28]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

torch.tensor(batched_ids)

ValueError: expected sequence of length 3 at dim 1 (got 2)

padding을 추가하여 문장들의 길이를 맞출 수 있다.

In [29]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]
torch.tensor(batched_ids)

tensor([[200, 200, 200],
        [200, 200, 100]])

#### Attention Mask

padding은 길이만 맞춰주는 무의미한 토큰이므로, 무시되어야 한다.  
∴ 모델에 \[200, 200\]과 \[200, 200, 100\]을 넣은 결과가 같아야 한다.

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print()
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)

tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


**Q**. 왜 두 결과가 다른가?  
**A**. attenntion_mask가 지정되지 않아, padding이 무시되지 않았기 때문이다.

In [31]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits) # attention_mask 입력 → 결과 동일

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


#### Truncation

문장의 인코딩을 잘라내야할 때
1. 모델이 입력으로 받을 수 있는 인코딩의 최대 길이보다 길 때
2. 특정 길이로 문장들을 잘라서 쓰고 싶을 때

In [32]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# 모델이 입력 최대 길이까지 자르기
model_inputs = tokenizer(sequences, truncation=True)

# 특정 길이까지 자르기
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

※ 상황별 Padding & Truncatinon 설정  
[HuggingFace - Documentations - Transformers - Conceptual Guides - Padding and Truncation](https://huggingface.co/docs/transformers/pad_truncation)

## 총정리

pipeline을 사용하지 않고 task 수행하기

In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt") # Tokenizer
output = model(**tokens) # Model

In [34]:
from pprint import pprint

# Post Processing
probs = torch.softmax(output.logits, dim=-1) # softmax
results = []

for prob in probs:
  pred_prob = torch.max(prob)
  pred_id = torch.argmax(prob)
  results.append({'label': model.config.id2label[int(pred_id)],
                  'score': float(pred_prob)})
pprint(results) # pipeline을 사용했을 때와 같은 결과

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'POSITIVE', 'score': 0.9994646906852722}]
